In [69]:
import psycopg2
# emet-dev.science.gc.ca
conn = psycopg2.connect(
    host="emet-dev.science.gc.ca",
    database="emet_zhen",
    user="guest",
    password="guest"
    )


In [70]:
import pandas as pd
db_cursor = conn.cursor()
	
# s = ""
# s = s + "SELECT"
# s = s + " table_schema"
# s = s +  ", table_name"
# s = s +  " FROM information_schema.tables"
# s = s +  " WHERE"
# s = s + " ("
# s = s + " table_schema = 'public'"
# s = s + " )"
# s = s + " ORDER BY table_schema, table_name;"

# db_cursor.execute(s)
# list_tables = db_cursor.fetchall()
# list_tables


In [66]:
# var_df = pd.read_sql_query("SELECT * FROM variable where code in ('12004','12006','11012')", conn)
var_df = pd.read_sql_query(f"SELECT id_var, description, mesure_unit FROM variable where code in  ('12004','12006','11012')", conn)
# df.to_excel('../variables.xlsx',index=False)

translation_table = str.maketrans("éàèùâêîôûç", "eaeuaeiouc")
def fix_text(d):
    return d.lower().replace(' ','_').translate(translation_table)

var_df['description'] = var_df['description'].map(fix_text)
var_df['mesure_unit'] = var_df['mesure_unit'].map(fix_text)
var_df.to_dict(orient='records')



[{'id_var': 11012,
  'description': 'vitesse_du_vent_a_10_metres',
  'mesure_unit': 'm/s'},
 {'id_var': 12004,
  'description': 'temperature_a__2_metres',
  'mesure_unit': 'celsius'},
 {'id_var': 12006,
  'description': 'temperature_du_point_de_rosee',
  'mesure_unit': 'celsius'}]

In [72]:
df = pd.read_sql_query("SELECT * FROM obs_list" conn)
# df.to_excel('../obs_types.xlsx',index=False)
df
# obs_databases = df.loc[~df.last_file.isna()].obs_table.unique().tolist()
# obs_databases

SyntaxError: invalid syntax (2850991535.py, line 1)

In [57]:
import numpy as np
elems_ids = [12004, 12006, 11012]
elems_ids_str = tuple(np.array(elems_ids).astype(str).tolist())
elems_ids_str


('12004', '12006', '11012')

In [79]:
from pprint import pprint
var_df = pd.read_sql_query(f"SELECT id_var, description FROM variable where id_source = 0", conn)
var_df['id_var'] = var_df['id_var'].astype(np.int32)
var_df.to_excel('vars.xlsx',index = False)

In [67]:
import pytz
import pandas as pd
import psycopg2
import numpy as np
# emet-dev.science.gc.ca

SEL_DATE = '201701'

REQUESTED_ELEM_IDS = [12004, 12006, 11012]

translation_table = str.maketrans("éàèùâêîôûç", "eaeuaeiouc")


def add_tz(d,tz = pytz.timezone('UTC')):
    return d.replace(tzinfo=tz).timestamp()


def fix_text(d):
    return d.lower().replace(' ','_').translate(translation_table)


conn = psycopg2.connect(
    host="emet-dev.science.gc.ca",
    database="emet_zhen",
    user="guest",
    password="guest"
    )

db_cursor = conn.cursor()


elems_ids_str = tuple(np.array(REQUESTED_ELEM_IDS).astype(str).tolist())


var_df = pd.read_sql_query(f"SELECT id_var, description FROM variable where code in {elems_ids_str}", conn)

var_df['description'] = var_df['description'].map(fix_text)

elems_list = var_df.to_dict(orient='records')


dfs = []

for elem in elems_list:
    dfs.append(pd.read_sql_query(f"SELECT date_valid as time, lat, lon, value AS {elem['description']} FROM obs.ade_synop_{SEL_DATE} WHERE id_var = {elem['id_var']}", conn))

ndf = dfs[0]

for i in range(1,len(dfs)):
    ndf = pd.merge(ndf, dfs[i], how='outer', on=['time', 'lat', 'lon'])


ndf['lat'] = ndf['lat'] / 1000000
ndf['lon'] = ndf['lon'] / 1000000


ndf['time'] = ndf['time'].map(add_tz)

ndf = ndf.astype(np.float32)
ndf = ndf.astype({'time':np.float64})

display(ndf)
display(ndf.describe())


time                             float64
lat                              float32
lon                              float32
vitesse_du_vent_a_10_metres      float32
temperature_a__2_metres          float32
temperature_du_point_de_rosee    float32
dtype: object

,time,lat,lon,vitesse_du_vent_a_10_metres,temperature_a__2_metres,temperature_du_point_de_rosee
0,1.483229e+09,-90.000000,0.000000,4.6,-24.799999,NaN
1,1.483229e+09,-85.199997,-87.879997,5.7,-11.100000,-14.800000
2,1.483229e+09,-84.000000,-79.500000,4.1,-11.100000,-14.500000
3,1.483229e+09,-82.769997,-13.050000,5.7,-17.299999,-20.799999
4,1.483229e+09,-80.370003,77.370003,1.5,NaN,NaN
...,...,...,...,...,...,...
3215524,1.485864e+09,-24.600000,24.670000,NaN,NaN,19.500000
3215525,1.485864e+09,41.930000,-82.519997,NaN,NaN,-2.300000
3215526,1.485864e+09,43.669998,-79.400002,NaN,NaN,-8.700000
3215527,1.485875e+09,55.970001,124.879997,NaN,NaN,-32.200001


,time,lat,lon,vitesse_du_vent_a_10_metres,temperature_a__2_metres,temperature_du_point_de_rosee
count,3.215529e+06,3.215529e+06,3.215529e+06,3.106377e+06,3.174560e+06,3.114247e+06
mean,1.484565e+09,3.600882e+01,3.087097e+01,3.495059e+00,2.389919e+00,-1.949228e+00
std,7.729479e+05,3.100759e+01,6.623660e+01,3.239329e+00,1.411964e+01,1.300630e+01
min,1.483229e+09,-9.000000e+01,-1.799800e+02,0.000000e+00,-8.000000e+01,-8.220000e+01
25%,1.483898e+09,3.228000e+01,6.120000e+00,1.000000e+00,-5.700000e+00,-9.000000e+00
50%,1.484568e+09,4.687000e+01,1.817000e+01,2.600000e+00,5.000000e-01,-2.400000e+00
75%,1.485238e+09,5.418000e+01,7.092000e+01,5.000000e+00,9.800000e+00,5.000000e+00
max,1.485904e+09,8.365000e+01,1.797500e+02,6.000000e+01,5.000000e+01,3.490000e+01


,time,lat,lon,v_12004,v_12006,v_11012
count,3.215529e+06,3.215529e+06,3.215529e+06,3.174560e+06,3.114247e+06,3.106377e+06
mean,1.484565e+09,3.599311e+01,3.087114e+01,2.389962e+00,-1.949226e+00,3.494680e+00
std,7.729477e+05,3.101008e+01,6.624390e+01,1.411952e+01,1.300637e+01,3.239661e+00
min,1.483229e+09,-9.000000e+01,-1.799800e+02,-8.000000e+01,-8.220000e+01,0.000000e+00
25%,1.483898e+09,3.228000e+01,6.120000e+00,-5.700000e+00,-9.000000e+00,1.000000e+00
50%,1.484568e+09,4.687000e+01,1.817000e+01,5.000000e-01,-2.400000e+00,2.600000e+00
75%,1.485238e+09,5.418000e+01,7.092000e+01,9.800000e+00,5.000000e+00,5.000000e+00
max,1.485904e+09,8.365000e+01,1.797500e+02,5.000000e+01,3.490000e+01,6.000000e+01


time       float64
lat        float32
lon        float32
v_12004    float32
v_12006    float32
v_11012    float32
dtype: object

In [44]:
ndf = ndf.astype({'time':np.float32,'lat':np.float32,'lon':np.float32,'v_12004':np.float32,'v_12006':np.float32,'v_11012':np.float32})
ndf.dtypes

time       float32
lat        float32
lon        float32
v_12004    float32
v_12006    float32
v_11012    float32
dtype: object

In [11]:

# df = pd.read_sql_query("SELECT date_valid,lat,lon,array_agg(ARRAY[id_var::INT,value::FLOAT]) FROM obs.ade_synop_202206 WHERE id_var IN (12004,12006,11012) GROUP BY date_valid,lat,lon", conn)
df = pd.read_sql_query("SELECT date_valid,lat,lon,value FROM obs.ade_synop_202206 WHERE id_var IN (12004) GROUP BY date_valid,lat,lon", conn)
df['lat'] = df['lat'] / 1000000
df['lon'] = df['lon'] / 1000000

def add_tz(d,tz = pytz.timezone('UTC')):
    return d.replace(tzinfo=tz)

df.rename(columns={'date_valid':'time'},inplace=True)

df['time'] = df['time'].map(add_tz)
df

DatabaseError: Execution failed on sql 'SELECT date_valid,lat,lon,value FROM obs.ade_synop_202206 WHERE id_var IN (12004) GROUP BY date_valid,lat,lon': column "ade_synop_202206.value" must appear in the GROUP BY clause or be used in an aggregate function
LINE 1: SELECT date_valid,lat,lon,value FROM obs.ade_synop_202206 WH...
                                  ^


In [10]:
for t in obs_databases:
    try :
        adf = pd.read_sql_query(f'SELECT * FROM {t}', conn)
        adf.to_excel(f'../emet/{t}.xlsx',index=False)
    except:
        pass

In [ ]:
db_cursor = conn.cursor()
	
s = ""
s = s + "SELECT"
s = s + " table_schema"
s = s +  ", table_name"
s = s +  " FROM information_schema.tables"
s = s +  " WHERE"
s = s + " ("
s = s + " table_schema = 'public'"
s = s + " )"
s = s + " ORDER BY table_schema, table_name;"

db_cursor.execute(s)
list_tables = db_cursor.fetchall()
list_tables
# for (_, t_name_table) in list_tables:
#     print(t_name_table)
#     q = f'SELECT * FROM {t_name_table};'
#     display(q)
#     db_cursor.execute(q)
#     results = db_cursor.fetchall()
#     display(results)
    